In [2]:
import requests
from bs4 import BeautifulSoup
import time        
import re
import io

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

In [3]:
# convert_pdf_to_txt by: karpathy / hynekcer
# @ https://stackoverflow.com/questions/5725278/how-do-i-use-pdfminer-as-a-library
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    fp.close()
    device.close()
    text = retstr.getvalue()
    retstr.close()
    return text

#a = convert_pdf_to_txt("bibliotecas/Guia_de_Bibliotecas.pdf")
b = convert_pdf_to_txt("bibliotecas/GuiaDeBibliotecasComunitarias.pdf")
original = b
cidades = original.split("MUNICÍPIO")

In [284]:
class municipio:
    def __init__(self, info):  
        self.nome = info[0]
        self.bibli = info[1]
        self.end = info[2]
        self.cep = info[3]
        self.hora = info[4]
        self.email = info[5]
        self.tel = info[6]
        self.resp = info[7]
        self.pag = 0
        
    def print(self):
        return(self.nome + "*" + self.bibli + "*" + self.end + \
             "*" + self.cep + "*" + self.hora + "*" + self.email + \
             "*" + self.tel + "*" + self.resp + "\n")

In [285]:
def get_info (cidade):
    nome = cidade[0]
    bibli = ""
    end = ""
    cep = ""
    hora = ""
    email = ""
    tel = ""
    resp = ""
    
    for i in range(len(cidade)):
        if (re.match(".*Biblioteca.*", cidade[i])):
            bibli = cidade[i].strip()
        if (re.match(".*End.*", cidade[i])):
            end = cidade[i + 1].strip()
        if (re.match(".*CEP.*", cidade[i])):
            cep = cidade[i][5:].strip()
        if (re.match(".*Hor.*", cidade[i])):
            hora = cidade[i + 1].strip()
        if (re.match(".*@.*", cidade[i])):
            email = cidade[i].strip()
        if (re.match(".*\(\d{2}\).*", cidade[i])):
            tel = cidade[i].strip()
        if (re.match(".*Resp.*", cidade[i])):
            resp = cidade[i + 1].strip()
    return nome, bibli, end, cep, hora, email, tel, resp

In [286]:
total = []
t = 0
for cidade in cidades:
    cidade = cidade.split("\n")
    cidade = [x for x in cidade if len(x) >= 3]
    if len(cidade) > 1:
        t += 1
        total.append(municipio(get_info(cidade)))
    
print(t) #checar se pegou todas as cidades

41


In [291]:
from unidecode import unidecode

count = 0
pag_inicial = 7
letra = "A"
for t in total:
    if (unidecode(t.nome[0]) != unidecode(letra)):
        letra = t.nome[0]
        count += 1
    t.pag = pag_inicial + count
    count += 1
    
for t in total:
    print(t.pag)

7
9
11
12
14
16
18
19
20
22
23
25
27
29
30
31
33
35
36
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
59
60


In [289]:
with open('bibliotecas/GuiaDeBibliotecasComunitarias.txt', 'w') as f:
    for t in total:
        f.write(t.nome + "*" + t.bibli + "*" + t.end + \
             "*" + t.cep + "*" + t.hora + "*" + t.email + \
             "*" + t.tel + "*" + t.resp + "*" + str(t.pag) + "\n")